In [0]:
import mysql.connector as con
from mysql.connector import Error
import pandas as pd
import numpy as np

In [0]:
#pip search mysql-connector | grep --color mysql-connector-python

mysql-connector-python (8.0.19)                       - MySQL driver written in Python
mysql-connector-python-rf (2.2.2)                     - MySQL driver written in Python
mysql-connector-python-dd (2.0.2)                     - MySQL driver written in Python


In [0]:
#pip install mysql-connector-python-rf

     |████████████████████████████████| 11.9MB 2.8MB/s 
  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp36-cp36m-linux_x86_64.whl size=249455 sha256=fa125adaef8ef2f58139c8737f51a934f69a7ae1fda4f889a4aec993170aa9da
  Stored in directory: /root/.cache/pip/wheels/87/58/fb/d95c84fad7e1bebfed324c13e107ebb08e1997c9226532859a
Successfully built mysql-connector-python-rf


In [0]:
#cnx = con.connect(user='student',passwd='student',host='cpanel.insaid.co',database='Capstone1')
#cur = cnx.cursor()
#cur.execute("desc gender_age_train");
#colnames = list()
#row = cur.fetchone()
#while row != None:
#  colnames.append(row[0])
#  row = cur.fetchone()

#cur1 = cnx.cursor()
#cur1.execute("select * from gender_age_train");
#datalist = list(cur1.fetchall())

#pf = pd.DataFrame(data=datalist,columns=colnames)
#pf

In [0]:
def database_connection(userId,pwd,hostname,dbinfo):
   cnx = con.connect(user=userId,passwd=pwd,host=hostname,database=dbinfo)
   return cnx

def fetch_columns(tablename,connection):
   colnames = list()
   datatypes = list()
   cursor = connection.cursor()
   query="desc " + tablename;
   cursor.execute(query)
   record = cursor.fetchone()
   while record != None:
      colnames.append(record[0])
      datatypes.append(record[1])
      record = cursor.fetchone()
   cursor.close()
   return colnames,datatypes

def fetch_records(tablename,connection):
   cursor = connection.cursor()
   query="select * from " + tablename
   cursor.execute(query)
   records = list(cursor.fetchall())
   cursor.close()
   return records
   

In [0]:
dataF = {}
def ExtractData(tablename,connection):
   columnNames = list()
   dataTypes = list()
   dataList = list()
   columnNames, dataTypes = fetch_columns(tablename, connection)
   dataList = fetch_records(tablename,connection)
   df = pd.DataFrame(data=dataList,columns=columnNames)
   print(columnNames)
   return df
   
def EstablishConnection(userId,pwd,host,db):   
   connection = database_connection(userId,pwd,host,db)
   try:
      if connection.is_connected():
         number = 0
         for table in TableList:
            dataF[number] = ExtractData(table,connection)
            number = number + 1
   except Error as e:
      print("ERROR while connection to Mysql", e)
   finally:
      if connection.is_connected():
         connection.close()
   return

TableList = ['gender_age_train','phone_brand_device_model']
EstablishConnection('student','student','cpanel.insaid.co','Capstone1')

['device_id', 'gender', 'age', 'group']
['device_id', 'phone_brand', 'device_model']


In [0]:
df = pd.DataFrame()
df = pd.concat(dataF, keys=None, ignore_index=True,)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,age,device_id,device_model,gender,group,phone_brand
0,35.0,-8076087639492063270,NaN,M,M32-38,NaN
1,35.0,-2897161552818060146,NaN,M,M32-38,NaN
2,35.0,-8260683887967679142,NaN,M,M32-38,NaN
3,30.0,-4938849341048082022,NaN,M,M29-31,NaN
4,30.0,245133531816851882,NaN,M,M29-31,NaN
...,...,...,...,...,...,...
162366,NaN,-4961458925928574032,荣耀畅玩4X,NaN,NaN,华为
162367,NaN,-8819817317449262375,荣耀6,NaN,NaN,华为
162368,NaN,-3358291377416935026,荣耀畅玩4,NaN,NaN,华为
162369,NaN,3282788959750981633,MI 2,NaN,NaN,小米


In [0]:
df = pd.DataFrame()
df = pd.merge(dataF[0], dataF[1], how='right', on='device_id')
dfok

,device_id,gender,age,group,phone_brand,device_model
0,-8076087639492063270,M,35.0,M32-38,小米,MI 2
1,-2897161552818060146,M,35.0,M32-38,小米,MI 2
2,-8260683887967679142,M,35.0,M32-38,小米,MI 2
3,-4938849341048082022,M,30.0,M29-31,三星,Galaxy S4
4,245133531816851882,M,30.0,M29-31,SUGAR,时尚手机
...,...,...,...,...,...,...
87721,-6699575201765951746,NaN,NaN,NaN,小米,MI 2S
87722,-8452552323787168430,NaN,NaN,NaN,努比亚,Z9 mini
87723,-1506492135756897958,NaN,NaN,NaN,小米,红米1S
87724,-1242294572141344487,NaN,NaN,NaN,三星,Galaxy Mega 5.8


In [0]:
eventsData = pd.DataFrame('events_data.csv')

In [0]:
dataF[1].phone_brand.unique()

array(['vivo', '小米', 'OPPO', '三星', '酷派', '联想 ', '华为', '奇酷', '魅族', '斐讯',
       '中国移动', 'HTC', '天语', '至尊宝', 'LG', '欧博信', '优米', 'ZUK', '努比亚', '惠普',
       '尼比鲁', '美图', '乡米', '摩托罗拉', '梦米', '锤子', '富可视', '乐视', '海信', '百立丰',
       '一加', '语信', '海尔', '酷比', '纽曼', '波导', '朵唯', '聆韵', 'TCL', '酷珀', '爱派尔',
       'LOGO', '青葱', '果米', '华硕', '昂达', '艾优尼', '康佳', '优购', '邦华', '赛博宇华',
       '黑米', 'Lovme', '先锋', 'E派', '神舟', '诺基亚', '普耐尔', '糖葫芦', '亿通', '欧新',
       '米奇', '酷比魔方', '蓝魔', '小杨树', '贝尔丰', '糯米', '米歌', 'E人E本', '西米', '大Q',
       '台电', '飞利浦', '唯米', '大显', '长虹', '维图', '青橙', '本为', '虾米', '夏新', '帷幄',
       '百加', 'SUGAR', '欧奇', '世纪星', '智镁', '欧比', '基伍', '飞秒', '德赛', '易派',
       '谷歌', '金星数码', '广信', '诺亚信', 'MIL', '白米', '大可乐', '宝捷讯', '优语', '首云',
       '瑞米', '瑞高', '沃普丰', '摩乐', '鲜米', '凯利通', '唯比', '欧沃', '丰米', '恒宇丰',
       '奥克斯', '西门子', '欧乐迪', 'PPTV'], dtype=object)

In [0]:
pip install py-translate

     |████████████████████████████████| 61kB 1.9MB/s 


In [0]:
dataF[1]


,device_id,phone_brand,device_model
0,1877775838486905855,vivo,Y13
1,-3766087376657242966,小米,V183
2,-6238937574958215831,OPPO,R7s
3,8973197758510677470,三星,A368t
4,-2015528097870762664,小米,红米Note2
...,...,...,...
87721,-4961458925928574032,华为,荣耀畅玩4X
87722,-8819817317449262375,华为,荣耀6
87723,-3358291377416935026,华为,荣耀畅玩4
87724,3282788959750981633,小米,MI 2


In [0]:
#import translate
#from translate import translator
!pip install googletrans
import googletrans
#translat = translator()

#dataF[1].phone_brand.apply(lambda x: translate --translit zh-tw en <<< x )

  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15777 sha256=b957165dc2e4821a4ff202a6a44f6bb01177506e0cf8c212d8ff52fb03a01681
  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [0]:
dataF[1].phone_brand

0        vivo
1          小米
2        OPPO
3          三星
4          小米
         ... 
87721      华为
87722      华为
87723      华为
87724      小米
87725      酷比
Name: phone_brand, Length: 87726, dtype: object

In [0]:
!pip install langdetect

     |████████████████████████████████| 1.0MB 2.8MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993459 sha256=374c56cde09f8c4e21c0f6c6931435d4b0f2a5a621796e10d4ba9bec427797af
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
import googletrans
from googletrans import Translator
from langdetect import detect
translator = Translator()
#dataF[1].phone_brand.apply(lambda x: translator.translate(x,dest='en',src='zh-tw') )
inputs = list(dataF[1].phone_brand.unique())
#inputs = ['三星','华为']
for x in inputs:
   lang = detect(x)
   if lang != 'en':
      translations = translator.translate(x,dest='en',src='zh-cn')
      print(translations.text)
#   print(translations)
#dataF[1].phone_brand

#translations = translator.translate(inputs,src='zh-cn',dest='en')
#translations
#type(inputs)
#for trans in translated:
#    print(f'{trans.origin} -> {trans.text}')

JSONDecodeError: ignored

In [0]:
pip install emoji

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=9afb11ff35ab63caa414cd626285c6d00353d66fde1e33ae0e5a9dfa284e0742
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
